In [ ]:
#Importing required Libraries
import csv  #csv operations
import os #renaming
import chardet #for encoding detetction
import requests
#import pandas as pd   #pandas for data frames
#trainingdata=pd.read_csv('/Users/russell/NTU/URECA/dataprocessor/CA-2el-Pt-An-PVK-GE-NiFoil-CuPd-1MKOH-0.1Gly.csv')
#print(trainingdata.shape)
#trainingdata.to_csv("train.txt", sep=",", index=False)

In [16]:
def download_psdata(dst_path="PSData.py"):
    url = "https://raw.githubusercontent.com/Russe11tham/dataprocessor/refs/heads/main/update-3"
    resp = requests.get(url, timeout=30)
    resp.raise_for_status()  # raise if download failed

    with open(dst_path, "wb") as f:
        f.write(resp.content)

    print(f"Downloaded PSData.py to {dst_path}")

In [17]:
#downloading from github
download_psdata()

from PSData import jparse
ps = jparse([input_file])# .pssession is in the same folder as this notebook

print(ps.experimentList)
print(ps.data.keys())

Downloaded PSData.py to PSData.py


NameError: name 'input_file' is not defined

In [ ]:
def ifcsv(input_file, output_file, nrows):
    # Detect encoding first
    with open(input_file, 'rb') as f:
        raw_data = f.read(10000)  # Sample for speed
        detected = chardet.detect(raw_data)
        encoding = detected['encoding']

    # Read all values into a flat list using detected encoding
    values = []
    with open(input_file, newline="", encoding=encoding) as f:
        reader = csv.reader(f)
        for row in reader:
            values.extend(row)

    # Write out nrows values per row as UTF-8
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        while values:
            writer.writerow(values[:nrows])
            values = values[nrows:]

    print(f"Output saved to {output_file} as UTF-8")

In [ ]:
def ifpssession(pssession_path, csv_path, experiment_name=None):
    """
    Convert a PalmSens .pssession file to CSV.
    If experiment_name is None, exports all experiments into one CSV with an 'experiment' column.
    """
    # Load the .pssession
    psdata = PSData()
    psdata.jparse(pssession_path)  # parses the pssession file

    # Decide which experiments to export
    experiments = psdata.experimentList  # list of experiment names
    if experiment_name is not None:
        if experiment_name not in experiments:
            raise ValueError(f"Experiment '{experiment_name}' not found. Available: {experiments}")
        experiments = [experiment_name]

    # Open CSV and write header
    with open(csv_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["experiment", "x", "y"])  # adjust column names as needed

        # Loop through experiments and dump x/y data
        for exp in experiments:
            data = psdata.data[exp]      # data object for this experiment
            x_vals = data.xvalues        # x‑axis values (e.g. potential)
            y_vals = data.yvalues        # y‑axis values (e.g. current)

            for x, y in zip(x_vals, y_vals):
                writer.writerow([exp, x, y])

    print(f"Saved CSV to {csv_path}")

In [ ]:
def iftxt(input_file, output_file, nrows):
    # Read text and split by comma
    with open(input_file, "r", encoding="utf-8") as f:
        text = f.read()

    values = [v.strip() for v in text.split(",") if v.strip()]

    # Write to CSV: every nrows values become one row
    with open(output_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        while values:
            writer.writerow(values[:nrows])
            values = values[nrows:]

In [ ]:
def renaming(old_name, finalname):
    new_name = f"{finalname}.csv"
    os.rename(old_name, new_name)

In [15]:
def main():
    print("Welcome to Photochemistry File Converter")
    print("file options: csv / txt / pssession")

    input_file = input("Enter current file name (with extension): ")
    filetype = input("Enter the type of file (csv/txt/pssession): ").lower()
    nrows = int(input("Enter the number of variables per row: "))
    finalname = input("Enter desired final file name (without .csv): ")

    output_tmp = "output.csv"

    print("DEBUG: filetype =", filetype)

    if filetype == "csv":
        print("DEBUG: in csv branch")
        ifcsv(input_file, output_tmp, nrows)
        renaming(output_tmp, finalname)

    elif filetype == "txt":
        print("DEBUG: in txt branch")
        iftxt(input_file, output_tmp, nrows)
        renaming(output_tmp, finalname)

    elif filetype == "pssession":
        print("DEBUG: in pssession branch")
        pssession_path = input_file
        csv_path = output_tmp
        ifpssession(pssession_path, csv_path, experiment_name=None)
        renaming(output_tmp, finalname)

    else:
        print("Invalid input, please try again.")

main()

Welcome to Photochemistry File Converter
file options: csv / txt / pssession


Enter current file name (with extension):  CA-2el-Pt-An-PVK-GE-NiFoil-CuPd-1MKOH-0.1Gly.pssession
Enter the type of file (csv/txt/pssession):  pssession
Enter the number of variables per row:  3
Enter desired final file name (without .csv):  realCAtest


DEBUG: filetype = pssession
DEBUG: in pssession branch


NameError: name 'ifpssession' is not defined